<a href="https://colab.research.google.com/github/cedalexandre/Few-Shot-Patch-Based-Training/blob/master/DEAMS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
###### INITIALISATION #####

resume = False

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  translator = Translator()
except: pass
!pip install ftfy==5.8

!apt-get -qq install ffmpeg
from google.colab import drive
drive.mount('/G', force_remount=True)
gdir = !ls /G/
gdir = '/G/%s/' % str(gdir[0])
%cd $gdir
work_dir = 'illustra'
work_dir = gdir + work_dir + '/'
import os
os.makedirs(work_dir, exist_ok=True)
%cd $work_dir

import os
import io
import time
import math
# from math import exp
import random
import imageio
import numpy as np
import PIL
# from skimage import exposure
from base64 import b64encode
import shutil
import moviepy, moviepy.editor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
!pip install git+https://github.com/Po-Hsun-Su/pytorch-ssim
import pytorch_ssim as ssim

%cd /content
!rm -rf aphantasia
#!git clone https://github.com/eps696/aphantasia
! git clone https://github.com/cedalexandre/aphantasia
%cd aphantasia/
from clip_fft import to_valid_rgb, fft_image, slice_imgs, checkout
from utils import pad_up_to, basename, file_list, img_list, img_read
from progress_bar import ProgressIPy as ProgressBar

clear_output()

if resume:
  resumed = files.upload()
  params_pt = list(resumed.values())[0]
  params_pt = torch.load(io.BytesIO(params_pt))

def read_pt(file):
  return torch.load(file).cuda()

def ema(base, next, step):
    scale_ma = 1. / (step + 1)
    return next * scale_ma + base * (1.- scale_ma)

def save_img(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def makevid(seq_dir, size=None):
  # out_sequence = seq_dir + '/%05d.jpg'
  out_video = seq_dir + '.mp4'
  # !ffmpeg -y -v quiet -i $out_sequence $out_video
  moviepy.editor.ImageSequenceClip(img_list(seq_dir), fps=25).write_videofile(out_video, verbose=False) # , ffmpeg_params=ffmpeg_params, logger=None
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

!nvidia-smi -L
print('\nDone!')

GPU 0: Tesla T4 (UUID: GPU-109dae3e-c1de-a8d1-5ee0-f92f375fd977)

Done!


In [9]:
multilang = False #param {type:"boolean"}
translate = False #param {type:"boolean"}
#uploaded = files.upload()

In [10]:
import pandas as pd
from google.colab import files 
import io
uploaded = files.upload()
todolist = pd.read_excel("dreams.xlsx")

Saving dreams.xlsx to dreams.xlsx


In [ ]:
for i in range(1) :
  sideX = 1280
  sideY = 720 
  duration =  todolist["DURATION"][i]

  #markdown > Config
  model = todolist["MODEL"][i] #'ViT-B/32' #param ['ViT-B/32', 'RN101', 'RN50x4', 'RN50']
  overscan = False
  keep = todolist["KEEP"][i]
  noise_scale = todolist["NOISE"][i]
  contrast = todolist["CONTRAST"][i]
  #markdown > Training
  steps = todolist["STEPS"][i] 
  samples = todolist["SAMPLES"][i]
  learning_rate = todolist["LEARNING RATE"][i]
  show_freq = todolist["FREQUENCY"][i]
  #markdown > Tricks
  diverse = todolist["DIVERSE"][i]
  expand = todolist["EXPAND"][i] 
  progressive_grow = todolist["PROGRESSIVE GROW"][i]
  fps = 25
  if multilang: model = 'ViT-B/32' # sbert model is trained with ViT

  model_clip, _ = clip.load(model)
  modsize = 288 if model == 'RN50x4' else 224
  xmem = {'RN50':0.5, 'RN50x4':0.16, 'RN101':0.33}
  if 'RN' in model:
    samples = int(samples * xmem[model])

  if diverse != 0:
    samples = int(samples * 0.5)
          
  norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

  text_file = todolist["TEST"][i]

  texts = texts = todolist["TEXT"][i].split('\n')
  texts = [tt.strip() for tt in texts if len(tt.strip())>0 and tt[0] != '#']
  print(' text file:', text_file)
  print(' total lines:', len(texts))

  workdir = os.path.join(work_dir, basename(text_file))
  workdir += '-%s' % model if 'RN' in model.upper() else ''
  !rm -rf $workdir
  os.makedirs(workdir, exist_ok=True)

  outpic = ipy.Output()
  outpic
    
  # make init
  global params_start, params_ema
  params_shape = [1, 3, sideY, sideX//2+1, 2]
  params_start = torch.randn(*params_shape).cuda() # random init
  params_ema = 0.

  if resume is True:
    # print(' resuming from', resumed)
    params, _ = fft_image([1, 3, sideY, sideX], resume = params_pt, sd=1.)
    if keep > 0:
      params_ema = params[0].detach()
    torch.save(params_pt, os.path.join(workdir, '000-start.pt'))
  else:
    torch.save(params_start, os.path.join(workdir, '000-start.pt'))

  torch.save(params_start, 'init.pt') # final init

  prev_enc = 0
  def process(txt, num):

    global params_start
    sd = 0.01
    if keep > 0: sd = keep + (1-keep) * sd
    params, image_f = fft_image([1, 3, sideY, sideX], resume='init.pt', sd=sd)
    image_f = to_valid_rgb(image_f)
    
    if progressive_grow is True:
      lr1 = learning_rate * 2
      lr0 = lr1 * 0.01
    else:
      lr0 = learning_rate
    optimizer = torch.optim.Adam(params, lr0)
      
    print(' ref text: ', txt)
    if translate:
      translator = Translator()
      txt = translator.translate(txt, dest='en').text
      print(' translated to:', txt)
    if multilang:
        model_lang = SentenceTransformer('clip-ViT-B-32-multilingual-v1').cuda()
        txt_enc = model_lang.encode([txt], convert_to_tensor=True, show_progress_bar=False).detach().clone()
        del model_lang
    else:
        txt_enc = model_clip.encode_text(clip.tokenize(txt).cuda()).detach().clone()

    out_name = '%03d-%s' % (num+1, txt.translate(str.maketrans(dict.fromkeys(list("\n',—|!?/:;\\"), ""))).replace(' ', '_').replace('"', ''))
    tempdir = os.path.join(workdir, out_name)
    !rm -rf $tempdir
    os.makedirs(tempdir, exist_ok=True)

    pbar = ProgressBar(steps) #  // save_freq
    for i in range(steps):
      loss = 0
      
      noise = noise_scale * torch.randn(1, 1, *params[0].shape[2:4], 1).cuda() if noise_scale > 0 else 0.
      img_out = image_f(noise)

      imgs_sliced = slice_imgs([img_out], samples, modsize, norm_in, overscan=overscan, micro=None)
      out_enc = model_clip.encode_image(imgs_sliced[-1])
      loss -= torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
      if diverse != 0:
        imgs_sliced = slice_imgs([image_f(noise)], samples, modsize, norm_in, overscan=overscan, micro=None)
        out_enc2 = model_clip.encode_image(imgs_sliced[-1])
        loss += diverse * torch.cosine_similarity(out_enc, out_enc2, dim=-1).mean()
        del out_enc2; torch.cuda.empty_cache()
      if expand > 0:
        global prev_enc
        if i > 0:
          loss += expand * torch.cosine_similarity(out_enc, prev_enc, dim=-1).mean()
        prev_enc = out_enc.detach()
      del img_out, imgs_sliced, out_enc; torch.cuda.empty_cache()

      if progressive_grow is True:
        lr_cur = lr0 + (i / steps) * (lr1 - lr0)
        for g in optimizer.param_groups: 
          g['lr'] = lr_cur

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      if i % show_freq == 0:
        with torch.no_grad():
          img = image_f(contrast=contrast).cpu().numpy()[0]
        save_img(img, os.path.join(tempdir, '%05d.jpg' % (i // show_freq)))
        outpic.clear_output()
        with outpic:
          display(Image('result.jpg'))
        del img

      pbar.upd()

    if keep > 0:
      global params_start, params_ema
      params_ema = ema(params_ema, params[0].detach(), num+1)
      torch.save((1-keep) * params_start + keep * params_ema, 'init.pt')

    torch.save(params[0], '%s.pt' % os.path.join(workdir, out_name))

  for i, txt in enumerate(texts):
      process(txt, i)

  vsteps = int(duration * fps / len(texts))
  tempdir = os.path.join(workdir, '_final')
  !rm -rf $tempdir
  os.makedirs(tempdir, exist_ok=True)

  print(' rendering complete piece')
  ptfiles = file_list(workdir, 'pt')
  pbar = ProgressBar(vsteps * len(ptfiles))
  for px in range(len(ptfiles)):
    params1 = read_pt(ptfiles[px])
    params2 = read_pt(ptfiles[(px+1) % len(ptfiles)])

    params, image_f = fft_image([1, 3, sideY, sideX], resume=params1, sd=1.)
    image_f = to_valid_rgb(image_f)

    for i in range(vsteps):
      with torch.no_grad():
        img = image_f((params2 - params1) * math.sin(1.5708 * i/vsteps)**2)[0].permute(1,2,0)
        img = torch.clip(img*255, 0, 255).cpu().numpy().astype(np.uint8)
      imageio.imsave(os.path.join(tempdir, '%05d.jpg' % (px * vsteps + i)), img)
      _ = pbar.upd()

  HTML(makevid(tempdir))

  #Downloading results
  import glob
  images = glob.glob('_out/*.jpg')
  for image in images : 
    print(image)
    files.download("_out/" + image)



 text file: monkey_1
 total lines: 1


Output()

 ref text:  monkey drinking beer
